In [1]:
# all import statements needed for the project, for example:
import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import sqlite3
import numpy as np
import matplotlib.animation as animation
import keplergl
from keplergl import KeplerGl
import statistics
import unittest

In [2]:
# any constants you might need, for example:
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Calculate the distance between the two coordinates
def calculate_distance(from_coord: list, to_coord: list) -> float:
    R = 6373.0
    lat1 = math.radians(from_coord[0])
    lon1 = math.radians(from_coord[1])
    lat2 = math.radians(to_coord[0])
    lon2 = math.radians(to_coord[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [4]:
# For the dataset that is not given the trip distance, calculate the distance using the given coordinate data and add it to the dataframe
def add_distance_column(dataframe: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    distance = []
    for index, row in dataframe.iterrows():
        distance.append(calculate_distance((row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])))
    dataframe['trip_distance'] = distance
    return dataframe

In [5]:
def find_taxi_parquet_urls() -> list:
    response = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    a = soup.find_all(lambda tag:'title' in tag.attrs and tag.attrs['title'] == "Yellow Taxi Trip Records")
    hrefs = [link.get('href') for link in a]
    hrefs_09 = list(filter(lambda href: '2009' in href, hrefs))
    hrefs_10 = list(filter(lambda href: '2010' in href, hrefs))
    hrefs_11 = list(filter(lambda href: '2011' in href, hrefs))
    hrefs_12 = list(filter(lambda href: '2012' in href, hrefs))
    hrefs_13 = list(filter(lambda href: '2013' in href, hrefs))
    hrefs_14 = list(filter(lambda href: '2014' in href, hrefs))
    hrefs_15 = list(filter(lambda href: '2015' in href, hrefs))
    for i in range(6):
        hrefs_15.pop()
    hrefs_t = hrefs_09 + hrefs_10 + hrefs_11 + hrefs_12 + hrefs_13 + hrefs_14 + hrefs_15
    return hrefs_t

In [6]:
find_taxi_parquet_urls()[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet'

In [ ]:
# Define a function that converts location to coordinates, and generate a dataframe
def convert_id_to_coord(df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    shapefile = gpd.read_file(r"C:\Users\Silvia\Documents\GitHub\4501FinalProject_Group14\taxi_zones\taxi_zones.shp")
    # Convert the geometry column in the shapefile into specific coordinates of latitude and longitude
    shapefile = shapefile.to_crs(4326)
    shapefile['latitude'] = shapefile['geometry'].centroid.y
    shapefile['longitude'] = shapefile['geometry'].centroid.x
    
    df = df
    df = df.loc[df["pulocationid"] <= 263]
    df = df.loc[df["pulocationid"] != 0]
    df = df.loc[df["dolocationid"] <= 263]
    df = df.loc[df["dolocationid"] != 0]
    # convert location IDs into longitude and latitude
    PUlongitude = []
    PUlatitude = []
    DOlongitude = []
    DOlatitude = []
    # convert the pickup location IDs into longitude and latitude
    for i in df['pulocationid']:
        PUlatitude.append(shapefile['latitude'][i-1])
        PUlongitude.append(shapefile['longitude'][i-1])
    for i in df['dolocationid']:
        DOlatitude.append(shapefile['latitude'][i-1])
        DOlongitude.append(shapefile['longitude'][i-1])
        
    df['pickup_longitude'] = PUlongitude
    df['pickup_latitude'] = PUlatitude
    df['dropoff_longitude'] = DOlongitude
    df['dropoff_latitude'] = DOlatitude
    # convert the drop off location IDs into longitude and latitude
    
    return df

In [ ]:
# Obtain taxi data and clean the data
def get_and_clean_month_taxi_data(url: str) -> pd.core.frame.DataFrame:

    df = pd.read_parquet(url)
    df.columns = df.columns.str.lower()
    df_taxi = pd.DataFrame()

    # keep necessary columns into a new dataframe
    if 'tpep_pickup_datetime' in df.columns:
        df=df.rename(columns = {'tpep_pickup_datetime':'pickup_datetime',
                                'tip_amount' : 'tip_amount'})
        df=convert_id_to_coord(df)
        
    elif 'trip_pickup_datetime' in df.columns:
        df=df.rename(columns = {'trip_pickup_datetime':'pickup_datetime', 
                                'start_lon': 'pickup_longitude',
                                'start_lat': 'pickup_latitude',
                                'end_lon': 'dropoff_longitude',
                                'end_lat': 'dropoff_latitude',
                                'tip_amt' : 'tip_amount'})
        
    df.drop(df.columns.difference(['pickup_datetime',
                                    'trip_distance', 
                                    'pickup_latitude', 
                                    'pickup_longitude', 
                                    'dropoff_latitude', 
                                    'dropoff_longitude',
                                   'tip_amount']), 1, inplace=True)
    
    df=df[df["pickup_longitude"] <= -73.717047]  
    df=df[df["pickup_longitude"] >= -74.242330]
    df=df[df["pickup_latitude"] >= 40.560445]
    df=df[df["pickup_latitude"] <= 40.908524]
    df=df[df["dropoff_longitude"] <= -73.717047]
    df=df[df["dropoff_longitude"] >= -74.242330]
    df=df[df["dropoff_latitude"] >= 40.560445]
    df=df[df["dropoff_latitude"] <= 40.908524]

    df = df.loc[df["pickup_datetime"] != 0.0]
    
    return df